# This code using NEO (Novel Expeditious Optimisation Algorithm)  for UR10e in Bin Picker applications

In [ ]:
# !pip3 install pybullet
# !pip3 install roboticstoolbox-python[collision]
# !pip install roboticstoolbox-python==1.1.0

## Include the library 

In [ ]:
import swift
import math as m
import spatialgeometry as sg
from spatialmath.base import *
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
import qpsolvers as qp
import cProfile
from spatialmath.pose3d import SE3

## Launch the simulation environment

In [ ]:
# Launch the simulator Swift
env = swift.Swift()
env.launch()

# Create a UR10e robot object
UR10e = rtb.models.UR10()

# Set joint angles to ready configuration
UR10e.q = [0.0, -1/2*m.pi, m.pi/2 , -m.pi/2, -m.pi/2, 0]

# Number of joint in the UR10e which we are controlling
n = 6


# Collisions
## s1: Base plate
s1 = sg.Cuboid(
    scale=[0.60, 1.2, 0.02],
    pose=sm.SE3(0.8, 0, 0.0))

# s2, s3 boundary plate front 1 and 2 robot
s2 = sg.Cuboid(
    scale=[0.60, 0.02, 0.3],
    pose=sm.SE3(0.8, 0.6, 0.15))

s3 = sg.Cuboid(
    scale=[0.60, 0.02, 0.3],
    pose=sm.SE3(0.8, -0.6, 0.15))

# s4, s5 boundary plate left and right robot
s4 = sg.Cuboid(
    scale=[0.02, 1.2, 0.3],
    pose=sm.SE3(0.5, 0, 0.15))

s5 = sg.Cuboid(
    scale=[0.02, 1.2, 0.3],
    pose=sm.SE3(1.1, 0, 0.15))

## s6: Center plate
s6 = sg.Cuboid(
    scale=[0.60, 0.02, 0.3],
    pose=sm.SE3(0.8, 0, 0.15))

## s7: Cylinder 
# s7 = sg.Cylinder(
#     radius =0.02, length =0.6,
#     pose=sm.SE3(0.8, 0, 0.28)*sm.SE3.RPY([0.0, m.pi/2, 0 ]))

## s7: Sphere
s7 = sg.Sphere(radius =0.07,
   pose=sm.SE3(0.55, 0, 0.25))
s7.v = [0.01, 0.00, 0.0, 0.1, 0.1, 0.1]
collisions = [s7]

# Make a target
target_1 = sg.Sphere(radius =0.01, pose=sm.SE3(0.7, -0.3, 0.30))
target_2 = sg.Sphere(radius =0.01, pose=sm.SE3(0.7, 0.3, 0.30))
# Add the Panda and shapes to the simulator
env.add(UR10e)
env.add(s1)
env.add(s2)
env.add(s3)
env.add(s4)
env.add(s5)
#env.add(s6)
env.add(s7)
env.add(target_1)
env.add(target_2)


# Set the desired end-effector pose to the location of target
Tep_1 = UR10e.fkine(UR10e.q)
Tep_1.A[:3, 3] = target_1.T[:3, 3] # get only position x y z
Tep_2 = UR10e.fkine(UR10e.q)
Tep_2.A[:3, 3] = target_2.T[:3, 3] # get only position x y z

#Show axis
# end-effector axes
ee_axes = sg.Axes(0.1)
# This pose can be either a spatialmat SE3 or a 4x4 ndarray
ee_axes.T = UR10e.fkine(UR10e.q)
# goal axes 1
goal_axes_1 = sg.Axes(0.1)
goal_axes_1.T = Tep_1
# goal axes 1=2
goal_axes_2 = sg.Axes(0.1)
goal_axes_2.T = Tep_2
# Add the axes to the environment
env.add(ee_axes)
env.add(goal_axes_1) 
env.add(goal_axes_2) 

## The main NEO code 

In [ ]:
def move(Tep):
    arrived = False
    while not arrived:
        # Change the Tep frequently
        #target.v = [0.03, 0.03, 0.0, 0.1, 0.1, 0.1]
        # The pose of the UR10e's end-effector
        Te = UR10e.fkine(UR10e.q)

        # Transform from the end-effector to desired pose
        eTep = Te.inv() * Tep

        # Spatial error
        e = np.sum(np.abs(np.r_[eTep.t, eTep.rpy() * np.pi / 180]))

        # Calulate the required end-effector spatial velocity for the robot
        # to approach the goal. Gain is set to 1.0
         # Specify the gain for the p_servo method
        gain = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
        # Calulate the required end-effector spatial velocity for the robot
        # to approach the goal. Gain is set to 1.0
        v, arrived = rtb.p_servo(Te, Tep, gain=gain, threshold=0.01)

        # Gain term (lambda) for control minimisation
        Y = 0.01

        # Quadratic component of objective function
        Q = np.eye(n + 6)

        # Joint velocity component of Q
        Q[:n, :n] *= Y

        # Slack component of Q
        Q[n:, n:] = (1 / e) * np.eye(6)

        # The equality contraints
        Aeq = np.c_[UR10e.jacobe(UR10e.q), np.eye(6)]
        beq = v.reshape((6,))

        # The inequality constraints for joint limit avoidance
        Ain = np.zeros((n + 6, n + 6))
        bin = np.zeros(n + 6)

        # The minimum angle (in radians) in which the joint is allowed to approach
        # to its limit
        ps = 0.2

        # The influence angle (in radians) in which the velocity damper
        # becomes active
        pi = 0.8

        # Form the joint limit velocity damper
        Ain[:n, :n], bin[:n] = UR10e.joint_velocity_damper(ps, pi, n)

        # For each collision in the scene
        for collision in collisions:

            # Form the velocity damper inequality contraint for each collision
            # object on the robot to the collision in the scene
            c_Ain, c_bin = UR10e.link_collision_damper(
                collision,
                UR10e.q[:n],
                0.8,
                0.06,
                1.5,
                start=UR10e.link_dict["shoulder_link"],
                end=UR10e.link_dict["ee_link"],
            )

            # If there are any parts of the robot within the influence distance
            # to the collision in the scene
            if c_Ain is not None and c_bin is not None:
                c_Ain = np.c_[c_Ain, np.zeros((c_Ain.shape[0], 6))]

                # Stack the inequality constraints
                Ain = np.r_[Ain, c_Ain]
                bin = np.r_[bin, c_bin]

        # Linear component of objective function: the manipulability Jacobian
        c = np.r_[-UR10e.jacobm(UR10e.q).reshape((n,)), np.zeros(6)]

        # The lower and upper bounds on the joint velocity and slack variable
        UR10e_qdlim = [2.175, 2.175, 2.175,  2.175, 2.61,  2.61]
        lb = -np.r_[UR10e_qdlim, 10 * np.ones(6)]
        ub = np.r_[UR10e_qdlim, 10 * np.ones(6)]

        # Solve for the joint velocities dq
        qd = qp.solve_qp(Q, c, Ain, bin, Aeq, beq, lb=lb, ub=ub, solver='osqp')

        # Apply the joint velocities to the UR10e
        UR10e.qd[:n] = qd[:n]

        # Step the simulator by 50 ms
        env.step(0.01)


In [ ]:
for i in range(4):
    move(Tep_1)
    move(Tep_2)